### DESEQ2 analysis of AD/PD dataset

In [1]:
rm(list=ls())
#load necessary libraries 
library(ggplot2)
library(DESeq2)
library("BiocParallel")
parallelFlag=TRUE
register(MulticoreParam(50))
library("IHW")
library("pheatmap")
library(sva)
library(limma)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, which

## Load data and design

In [2]:
#load ATAC-seq raw read counts
data=read.table('../adpd.atac.idr.counts.txt.gz',header=TRUE,sep='\t')
#concatenate chrom/start/end columns values to server as rownames for the dataframe of the form chrom_start_end 
rownames(data)=paste(data$chrom,data$start,data$end,sep="_")
data$chrom=NULL
data$start=NULL
data$end=NULL

data=data[rowSums(data)>0,]


In [7]:
head(data)

,ADAD_CAUD_00_0281,ADAD_CAUD_00_0387,ADAD_CAUD_01_0164,ADAD_CAUD_01_1400,ADAD_CAUD_06_0194,ADAD_CAUD_06_1486,ADAD_CAUD_07_0787,ADAD_CAUD_12829xx,ADAD_CAUD_23156xx,ADAD_CAUD_24281xx,⋯,LRRK_MDTG_01_39,LRRK_MDTG_04_10,LRRK_MDTG_10_37,LRRK_MDTG_13_60,LRRK_PTMN_01_39,LRRK_PTMN_04_10,LRRK_PTMN_10_37,LRRK_PTMN_13_60,LRRK_SUNI_04_10,LRRK_SUNI_10_37
chr1_10015_10231,12,16,22,12,14,20,12,22,16,10,⋯,3,2,14,18,0,4,12,9,22,26
chr1_181363_181563,1,6,4,2,10,6,1,9,1,3,⋯,1,5,2,14,0,4,7,4,8,18
chr1_183716_183916,7,4,4,12,2,10,19,14,8,10,⋯,1,19,10,21,3,16,8,6,20,13
chr1_184083_184283,11,8,4,11,2,19,20,27,6,18,⋯,1,20,26,25,3,22,18,11,26,24
chr1_184370_184570,7,6,6,0,6,9,6,11,5,9,⋯,5,16,11,18,1,13,7,4,23,15
chr1_190744_191148,84,95,136,110,43,163,97,186,66,158,⋯,6,70,122,46,8,78,46,36,79,93


In [4]:
#load the metadata
batches=read.table("../batches.filtered.csv",header=TRUE,sep='\t')

In [8]:
tail(batches)

,Sample,NewName,Type,TypeMod,Region,RegionMod,Cohort,TissueCenter,Batch,Gender,expired_age,PMI,ApoE
407,LRRK_PTMN_01_39,PD_01_39_LRRK_PTMN_X025_S12_L022_B1_T2_P046,LRRK,ADPD,PTMN,PTMN,PD,UA,PD_X025,Male,85,2.00,3_4
408,LRRK_PTMN_04_10,PD_04_10_LRRK_PTMN_X026_S10_L041_B1_T1_P050,LRRK,ADPD,PTMN,PTMN,PD,UA,PD_X026,Male,77,1.66,3_3
409,LRRK_PTMN_10_37,PD_10_37_LRRK_PTMN_X015_S05_L033_B1_T1_P026,LRRK,ADPD,PTMN,PTMN,PD,UA,PD_X015,Female,84,5.00,3_3
410,LRRK_PTMN_13_60,PD_13_60_LRRK_PTMN_X007_S07_L061_B1_T1_P010,LRRK,ADPD,PTMN,PTMN,PD,UA,PD_X007,Male,89,3.82,3_3
411,LRRK_SUNI_04_10,PD_04_10_LRRK_SUNI_X022_S06_L035_B1_T1_P043,LRRK,ADPD,SUNI,SUNI,PD,UA,PD_X022,Male,77,1.66,3_3
412,LRRK_SUNI_10_37,PD_10_37_LRRK_SUNI_X010_S11_L045_B1_T1_P016,LRRK,ADPD,SUNI,SUNI,PD,UA,PD_X010,Female,84,5.00,3_3


In [9]:
head(batches)

Sample,NewName,Type,TypeMod,Region,RegionMod,Cohort,TissueCenter,Batch,Gender,expired_age,PMI,ApoE
ADAD_CAUD_00_0281,ADAD_CAUD_00_0281_X008_S02_L051_B1_T1_P018,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X008,Male,61,16.0,3_3
ADAD_CAUD_00_0387,ADAD_CAUD_00_0387_X017_S14_L075_B1_T1_P045,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X017,Female,52,NA,3_4
ADAD_CAUD_01_0164,ADAD_CAUD_01_0164_X014_S04_L055_B1_T1_P029,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X014,Male,58,24.0,2_3
ADAD_CAUD_01_1400,ADAD_CAUD_01_1400_X011_S03_L053_B1_T1_P024,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X011,Male,55,24.0,3_4
ADAD_CAUD_06_0194,ADAD_CAUD_06_0194_X004_S11_L045_B1_T1_P007,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X004,Male,64,19.0,3_3
ADAD_CAUD_06_1486,ADAD_CAUD_06_1486_X007_S09_L041_B1_T1_P016,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X007,Male,60,20.5,3_3


In [20]:
Grouping <- factor(paste0(batches$Cohort,".",batches$RegionMod, ".", batches$TypeMod))
batches$Grouping=Grouping

In [21]:
#SVA can't handle NA values, so we have no choice but to interpolate to the mode for missing entries in PMI & ApoE 
batches$ApoE[is.na(batches$ApoE)]='3_3'
batches$PMI[is.na(batches$PMI)]=mean(na.omit(batches$PMI))

In [22]:
mod0=model.matrix(~1,data=batches)
mod1=model.matrix(~Grouping,data=batches)#+Gender+expired_age+PMI+TissueCenter,data=batches)#+Cohort+ApoE+Batch,data=batches)


In [27]:
sva.obj=svaseq(as.matrix(data),mod1,mod0,vfilter=10000,n.sv=10)#,n.sv = n.sv,vfilter=10000)

Number of significant surrogate variables is:  10 
Iteration (out of 5 ):1  2  3  4  5  

In [32]:
batches=cbind(batches,sur_var)

In [33]:
head(batches)

Sample,NewName,Type,TypeMod,Region,RegionMod,Cohort,TissueCenter,Batch,Gender,⋯,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10
ADAD_CAUD_00_0281,ADAD_CAUD_00_0281_X008_S02_L051_B1_T1_P018,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X008,Male,⋯,0.03570277,-0.031113562,0.02753915,-0.01717322,0.021097156,-0.04552970,0.02002344,-0.0458679587,0.016958965,-0.022143265
ADAD_CAUD_00_0387,ADAD_CAUD_00_0387_X017_S14_L075_B1_T1_P045,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X017,Female,⋯,-0.06477943,0.026740112,-0.01944008,0.04342368,0.031749792,0.03215059,-0.05265180,0.0347037689,0.026254001,-0.038928574
ADAD_CAUD_01_0164,ADAD_CAUD_01_0164_X014_S04_L055_B1_T1_P029,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X014,Male,⋯,0.03926742,0.027725245,0.02702905,0.05526815,-0.007170824,0.01868300,-0.01694325,0.0001110205,0.065153926,0.074567376
ADAD_CAUD_01_1400,ADAD_CAUD_01_1400_X011_S03_L053_B1_T1_P024,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X011,Male,⋯,0.04312626,-0.006713843,-0.07615340,0.02694130,0.027017233,-0.03818775,0.09586232,-0.0017419997,0.006962824,-0.028291514
ADAD_CAUD_06_0194,ADAD_CAUD_06_0194_X004_S11_L045_B1_T1_P007,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X004,Male,⋯,0.01811433,-0.057447059,0.01477661,0.04017166,-0.017050607,-0.03153783,-0.02427820,-0.0027016054,-0.046433264,0.007225739
ADAD_CAUD_06_1486,ADAD_CAUD_06_1486_X007_S09_L041_B1_T1_P016,ADAD,ADAD,CAUD,CAUD,AD,UW,AD_X007,Male,⋯,0.05533167,0.029248385,-0.04837718,-0.05911273,-0.009180071,-0.07248935,-0.06806201,0.0893697304,0.004394672,-0.087692486


## Run SVA-seq 

## Create the DESeq2 Object

In [34]:
#Create DESeq object
dds <- DESeqDataSetFromMatrix(countData = data,
                              colData = batches,
                              design = ~Grouping+Gender+expired_age+PMI+ApoE+X1+X2+X3+X4+X5+X6+X7+X8+X9+X10)


  the design formula contains a numeric variable with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function


## Differential Accessibility Operation

In [7]:
#Run the differential analysis
dds <- DESeq(dds,parallel = TRUE)

estimating size factors
estimating dispersions
gene-wise dispersion estimates: 50 workers
mean-dispersion relationship
final dispersion estimates, fitting model and testing: 50 workers
-- replacing outliers and refitting for 4 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing


In [ ]:
res=results(dds)
summary(res)

res=results(dds,independentFiltering=FALSE)
summary(res)

res=results(dds,filterFun = ihw)
summary(res)

In [ ]:
comparisons=c("pd_caud_adpd_vs_lopd",
    "pd_caud_adpd_vs_ctrl",
    "pd_caud_lopd_vs_ctrl",
    "pd_hipp_adpd_vs_lopd",
    "pd_hipp_adpd_vs_ctrl",
    "pd_hipp_lopd_vs_ctrl",
    "pd_mdfg_adpd_vs_lopd",
    "pd_mdfg_adpd_vs_ctrl",
    "pd_mdfg_lopd_vs_ctrl",
    "pd_ptmn_adpd_vs_lopd",
    "pd_ptmn_adpd_vs_ctrl",
    "pd_ptmn_lopd_vs_ctrl",
    "pd_smtg_adpd_vs_lopd",
    "pd_smtg_adpd_vs_ctrl",
    "pd_smtg_lopd_vs_ctrl",
    "pd_suni_adpd_vs_lopd",
    "pd_suni_adpd_vs_ctrl",
    "pd_suni_lopd_vs_ctrl")


In [ ]:
contrasts=list(c("Grouping_PD.CAUD.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.CAUD.LOPD_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.CAUD.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.CAUD.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.CAUD.LOPD_vs_AD.CAUD.ADAD","Grouping_PD.CAUD.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.HIPP.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.HIPP.LOPD_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.HIPP.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.HIPP.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.HIPP.LOPD_vs_AD.CAUD.ADAD","Grouping_PD.HIPP.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.MDFG.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.MDFG.LOPD_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.MDFG.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.MDFG.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.MDFG.LOPD_vs_AD.CAUD.ADAD","Grouping_PD.MDFG.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.PTMN.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.PTMN.LOPD_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.PTMN.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.PTMN.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.PTMN.LOPD_vs_AD.CAUD.ADAD","Grouping_PD.PTMN.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.SMTG.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.SMTG.LOPD_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.SMTG.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.SMTG.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.SMTG.LOPD_vs_AD.CAUD.ADAD","Grouping_PD.SMTG.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.SUNI.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.SUNI.LOPD_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.SUNI.ADPD_vs_AD.CAUD.ADAD","Grouping_PD.SUNI.CTRL_vs_AD.CAUD.ADAD"),
            c("Grouping_PD.SUNI.LOPD_vs_AD.CAUD.ADAD","Grouping_PD.SUNI.CTRL_vs_AD.CAUD.ADAD"))

In [ ]:
pval_thresh=0.05
lfc_thresh=1

In [ ]:
##get the results for the various contrasts 
numcomparisons=length(contrasts)
for(i in seq(1,numcomparisons))
{
 res=results(dds,contrast=contrasts[i],parallel=TRUE) 
 res$logPadj=-1*log10(res$padj)
 res=as.data.frame(res)
 res=na.omit(res)
 res$sig=res$padj<=pval_thresh & abs(res$log2FoldChange)>lfc_thresh
    
 #extract the differential peaks 
 sigsubset=res[res$sig==TRUE,]
 sig=nrow(sigsubset)
 up=sum(sigsubset$log2FoldChange>0)
 down=sum(sigsubset$log2FoldChange<0)
 curtitle=paste(comparisons[i],'\n','sig:',sig,'\n','up:',up,'\n','down:',down,'\n')   
 print(curtitle)
    
 #generate a volcano plot 
 png(paste("sva_volcano_diff",comparisons[i],".png",sep=""))
 print(ggplot(data=res,
               aes(y=res$logPadj,x=res$log2FoldChange,color=res$sig))+
               geom_point(alpha=0.1)+
               xlab("log2(FC)")+
               ylab("-log10(pval)")+
               theme_bw()+
               scale_color_manual(values=c("#000000","#FF0000"))+
               ggtitle(curtitle))
  dev.off() 
  #write differential peaks to a TSV file 
  write.table(sigsubset,file=paste("sva_diff_",comparisons[i],".tsv",sep=""),
              quote=FALSE,sep='\t',row.names = TRUE,col.names = TRUE)
}

In [ ]:
## Repeat analysis with Type and Region 

Grouping <- factor(paste0(batches$Cohort,".",batches$Region, ".", batches$Type))
batches$Grouping=Grouping


In [ ]:
#Create DESeq object
dds2 <- DESeqDataSetFromMatrix(countData = data,
                              colData = batches,
                              design = ~Grouping+Gender+expired_age+PMI+ApoE+X1+X2+X3+X4+X5+X6+X7+X8+X9+X10)


In [ ]:
#Run the differential analysis
dds2 <- DESeq(dds2,parallel = TRUE)

In [ ]:
comparisons=c("pd_caud_gba1_vs_lrrk",
    "pd_caud_gba1_vs_spor",
    "pd_caud_gba1_vs_ctrl",
    "pd_caud_lrrk_vs_spor",
    "pd_caud_lrrk_vs_ctrl",
    "pd_caud_spor_vs_ctrl",
    "pd_hipp_gba1_vs_lrrk",
    "pd_hipp_gba1_vs_spor",
    "pd_hipp_gba1_vs_ctrl",
    "pd_hipp_lrrk_vs_spor",
    "pd_hipp_lrrk_vs_ctrl",
    "pd_hipp_spor_vs_ctrl",
    "pd_mdfg_gba1_vs_lrrk",
    "pd_mdfg_gba1_vs_spor",
    "pd_mdfg_gba1_vs_ctrl",
    "pd_mdfg_lrrk_vs_spor",
    "pd_mdfg_lrrk_vs_ctrl",
    "pd_mdfg_spor_vs_ctrl",
    "pd_mdtg_gba1_vs_lrrk",
    "pd_mdtg_gba1_vs_spor",
    "pd_mdtg_gba1_vs_ctrl",
    "pd_mdtg_lrrk_vs_spor",
    "pd_mdtg_lrrk_vs_ctrl",
    "pd_mdtg_spor_vs_ctrl",
    "pd_ptmn_gba1_vs_lrrk",
    "pd_ptmn_gba1_vs_spor",
    "pd_ptmn_gba1_vs_ctrl",
    "pd_ptmn_lrrk_vs_spor",
    "pd_ptmn_lrrk_vs_ctrl",
    "pd_ptmn_spor_vs_ctrl",
    "pd_suni_gba1_vs_lrrk",
    "pd_suni_gba1_vs_spor",
    "pd_suni_gba1_vs_ctrl",
    "pd_suni_lrrk_vs_spor",
    "pd_suni_lrrk_vs_ctrl",
    "pd_suni_spor_vs_ctrl")


In [ ]:
contrasts=list(c("Grouping_PD.CAUD.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.CAUD.LRRK_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.CAUD.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.CAUD.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.CAUD.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.CAUD.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.CAUD.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.CAUD.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.CAUD.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.CAUD.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.CAUD.SPOR_vs_AD.CAUD.ADAD","Grouping_PD.CAUD.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.HIPP.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.HIPP.LRRK_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.HIPP.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.HIPP.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.HIPP.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.HIPP.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.HIPP.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.HIPP.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.HIPP.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.HIPP.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.HIPP.SPOR_vs_AD.CAUD.ADAD","Grouping_PD.HIPP.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDFG.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.MDFG.LRRK_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDFG.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.MDFG.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDFG.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.MDFG.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDFG.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.MDFG.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDFG.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.MDFG.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDFG.SPOR_vs_AD.CAUD.ADAD","Grouping_PD.MDFG.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDTG.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.MDTG.LRRK_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDTG.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.MDTG.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDTG.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.MDTG.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDTG.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.MDTG.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDTG.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.MDTG.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.MDTG.SPOR_vs_AD.CAUD.ADAD","Grouping_PD.MDTG.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.PTMN.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.PTMN.LRRK_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.PTMN.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.PTMN.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.PTMN.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.PTMN.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.PTMN.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.PTMN.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.PTMN.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.PTMN.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.PTMN.SPOR_vs_AD.CAUD.ADAD","Grouping_PD.PTMN.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.SUNI.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.SUNI.LRRK_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.SUNI.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.SUNI.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.SUNI.GBA1_vs_AD.CAUD.ADAD","Grouping_PD.SUNI.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.SUNI.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.SUNI.SPOR_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.SUNI.LRRK_vs_AD.CAUD.ADAD","Grouping_PD.SUNI.CTRL_vs_AD.CAUD.ADAD"),
    c("Grouping_PD.SUNI.SPOR_vs_AD.CAUD.ADAD","Grouping_PD.SUNI.CTRL_vs_AD.CAUD.ADAD"))

In [ ]:
##get the results for the various contrasts 
numcomparisons=length(contrasts)
for(i in seq(1,numcomparisons))
{
 res=results(dds2,contrast=contrasts[i],parallel=TRUE) 
 res$logPadj=-1*log10(res$padj)
 res=as.data.frame(res)
 res=na.omit(res)
 res$sig=res$padj<=pval_thresh & abs(res$log2FoldChange)>lfc_thresh
    
 #extract the differential peaks 
 sigsubset=res[res$sig==TRUE,]
 sig=nrow(sigsubset)
 up=sum(sigsubset$log2FoldChange>0)
 down=sum(sigsubset$log2FoldChange<0)
 curtitle=paste(comparisons[i],'\n','sig:',sig,'\n','up:',up,'\n','down:',down,'\n')   
 print(curtitle)
    
 #generate a volcano plot 
 png(paste("expanded_volcano_diff",comparisons[i],".png",sep=""))
 print(ggplot(data=res,
               aes(y=res$logPadj,x=res$log2FoldChange,color=res$sig))+
               geom_point(alpha=0.1)+
               xlab("log2(FC)")+
               ylab("-log10(pval)")+
               theme_bw()+
               scale_color_manual(values=c("#000000","#FF0000"))+
               ggtitle(curtitle))
  dev.off() 
  #write differential peaks to a TSV file 
  write.table(sigsubset,file=paste("expanded_diff_",comparisons[i],".tsv",sep=""),
              quote=FALSE,sep='\t',row.names = TRUE,col.names = TRUE)
}

In [ ]:
#store dds object so it can be loaded readily in the future 
save(dds,dds2 file = "DESEQ2.model.withSVA.RData")